In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim

import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)


REBUILD_DATA = False # set to true to one once, then back to false unless you want to change something in your training data.
training_data = []

class Labeling():
    
    #Labels = {C_major : 0 , Cs_major : 1, D_major : 2, Eb_major : 3, E_major : 4, F_major : 5, Fs_major : 6, G_major : 7, Ab_major : 8, A_major : 9, Bb_major : 10, B_major : 11, 
    #          C_major : 12 , C_minor : 13, D_minor : 14, Eb_minor : 15, E_minor : 16, F_minor : 17, Fs_minor : 18, G_minor : 19, Ab_minor : 20, A_minor : 21, Bb_minor : 22, B_minor : 23
    #         }
    training_data = []
    
    
    def make_training_data():
        path = "C:\spects"
        #path = "C:/outs/spects8192"
        os.chdir(path)
        spectrograms = os.listdir()
        key_count = [0] * 24
        annotations = open("C:/Users/Michalis Zeakis/Desktop/university/ptyxiaki/Annotations2090.txt", "r")
        count = 0
        
        while True: 
            line = annotations.readline()
            if not line:
                print(key_count , count)
                for keys in key_count :
                    print(((keys/len(training_data)))*100)
                break;
            
            count = count + 1
            file, key = line.split(' ', 1)
            key = key.strip()
            #print(key)
            file = file + ".LOFI.png"
            spect_path = os.path.join(path, file)
            #spect_path = cv2.imread(spect_path)
            spect_path = cv2.imread(spect_path, cv2.IMREAD_GRAYSCALE)
            
            if spect_path is None : 
                continue;
            
            
            if key == 'C major' :
                if key_count[0] >600:
                    continue;
                training_data.append([np.array(spect_path), torch.LongTensor([0])])
                key_count[0] = key_count[0] + 1
            elif key == 'C# major' :
                if key_count[1] >600:
                    continue;
                training_data.append([np.array(spect_path), torch.LongTensor([1])])   #|
                key_count[1] = key_count[1] + 1                                       #|
            elif key == 'Db major' :
                if key_count[1] >600:
                    continue;                                                         #| same key
                training_data.append([np.array(spect_path), torch.LongTensor([1])])   #|
                key_count[1] = key_count[1] + 1                                       #|
            elif key == 'D major' :
                if key_count[2] >600:
                    continue;
                training_data.append([np.array(spect_path), torch.LongTensor([2])])
                key_count[2] = key_count[2] + 1
            elif key == 'D# major' :
                if key_count[3] >600:
                    continue;
                training_data.append([np.array(spect_path), torch.LongTensor([3])])  #|
                key_count[3] = key_count[3] + 1
            elif key == 'Eb major' :
                if key_count[3] >600:
                    continue;                                                        #| same key
                training_data.append([np.array(spect_path), torch.LongTensor([3])])  #|
                key_count[3] = key_count[3] + 1
            elif key == 'E major' :
                if key_count[4] >600:
                    continue;
                training_data.append([np.array(spect_path), torch.LongTensor([4])])
                key_count[4] = key_count[4] + 1
            elif key == 'F major' :
                if key_count[5] >600:
                    continue;
                training_data.append([np.array(spect_path), torch.LongTensor([5])])
                key_count[5] = key_count[5] + 1
            elif key == 'F# major' :
                if key_count[6] >600:
                    continue;
                training_data.append([np.array(spect_path), torch.LongTensor([6])])  #|
                key_count[6] = key_count[6] + 1
            elif key == 'Gb major' :      
                if key_count[6] >600:
                    continue;                                                        #| same key
                training_data.append([np.array(spect_path), torch.LongTensor([6])])  #|
                key_count[6] = key_count[6] + 1
            elif key == 'G major' :
                if key_count[7] >600:
                    continue;
                training_data.append([np.array(spect_path), torch.LongTensor([7])])
                key_count[7] = key_count[7] + 1
            elif key == 'G# major' :
                if key_count[8] >600:
                    continue;
                training_data.append([np.array(spect_path),torch.LongTensor([8])])  #|
                key_count[8] = key_count[8] + 1
            elif key == 'Ab major' :   
                if key_count[8] >600:
                    continue;                                                       #| same key
                training_data.append([np.array(spect_path),torch.LongTensor([8])])  #|
                key_count[8] = key_count[8] + 1
            elif key == 'A major' :
                if key_count[9] > 600:
                    continue;
                training_data.append([np.array(spect_path), torch.LongTensor([9])])
                key_count[9] = key_count[9] + 1
            elif key == 'A# major' :
                if key_count[10] >600:
                    continue;
                training_data.append([np.array(spect_path),torch.LongTensor([10])]) #|
                key_count[10] = key_count[10] + 1
            elif key == 'Bb major' :    
                if key_count[10] >600:
                    continue;                                                        #| same key
                training_data.append([np.array(spect_path), torch.LongTensor([10])]) #|
                key_count[10] = key_count[10] + 1
            elif key == 'B major' :
                if key_count[11] >600:
                    continue;
                training_data.append([np.array(spect_path), torch.LongTensor([11])])
                key_count[11] = key_count[11] + 1
            elif key == 'C minor' :
                if key_count[12] >600:
                    continue;
                training_data.append([np.array(spect_path), torch.LongTensor([12])])
                key_count[12] = key_count[12] + 1
            elif key == 'C# minor' :
                if key_count[13] >600:
                    continue;
                training_data.append([np.array(spect_path), torch.LongTensor([13])])   #|
                key_count[13] = key_count[13] + 1
            elif key == 'Db minor' :
                if key_count[13] >600:
                    continue;                                                          #| same key
                training_data.append([np.array(spect_path), torch.LongTensor([13])])   #|
                key_count[13] = key_count[13] + 1
            elif key == 'D minor' :
                if key_count[14] >600:
                    continue;
                training_data.append([np.array(spect_path), torch.LongTensor([14])])
                key_count[14] = key_count[14] + 1
            elif key == 'D# minor' :
                if key_count[15] >600:
                    continue;
                training_data.append([np.array(spect_path), torch.LongTensor([15])])  #|
                key_count[15] = key_count[15] + 1
            elif key == 'Eb minor' :
                if key_count[15] >600:
                    continue;                                                         #| same key
                training_data.append([np.array(spect_path), torch.LongTensor([15])])  #|
                key_count[15] = key_count[15] + 1
            elif key == 'E minor' :
                if key_count[16] >600:
                    continue;
                training_data.append([np.array(spect_path), torch.LongTensor([16])])
                key_count[16] = key_count[16] + 1
            elif key == 'F minor' :
                if key_count[17] >600:
                    continue;
                training_data.append([np.array(spect_path),torch.LongTensor([17])])
                key_count[17] = key_count[17] + 1
            elif key == 'F# minor' :
                if key_count[18] >600:
                    continue;
                training_data.append([np.array(spect_path), torch.LongTensor([18])])  #|
                key_count[18] = key_count[18] + 1
            elif key == 'Gb minor' :
                if key_count[18] >600:
                    continue;                                                         #| same key
                training_data.append([np.array(spect_path), torch.LongTensor([18])])  #|
                key_count[18] = key_count[18] + 1
            elif key == 'G minor' :
                if key_count[19] >600:
                    continue;
                training_data.append([np.array(spect_path), torch.LongTensor([19])])
                key_count[19] = key_count[19] + 1
            elif key == 'G# minor' :
                if key_count[20] >600:
                    continue;
                training_data.append([np.array(spect_path), torch.LongTensor([20])])  #|
                key_count[20] = key_count[20] + 1
            elif key == 'Ab minor' :
                if key_count[20] >600:
                    continue;                                                         #| same key
                training_data.append([np.array(spect_path), torch.LongTensor([20])])  #|
                key_count[20] = key_count[20] + 1
            elif key == 'A minor' :
                if key_count[21] >600:
                    continue;
                training_data.append([np.array(spect_path), torch.LongTensor([21])])
                key_count[21] = key_count[21] + 1
            elif key == 'A# minor' :
                if key_count[22] > 600:
                    continue;
                training_data.append([np.array(spect_path), torch.LongTensor([22])]) #|
                key_count[22] = key_count[22] + 1
            elif key == 'Bb minor' :
                if key_count[22] > 600:
                    continue;                                                        #| same key
                training_data.append([np.array(spect_path), torch.LongTensor([22])]) #|
                key_count[22] = key_count[22] + 1
            elif key == 'B minor' :
                if key_count[23] > 600:
                    continue;
                training_data.append([np.array(spect_path), torch.LongTensor([23])])
                key_count[23] = key_count[23] + 1

        
        np.random.shuffle(training_data)
        #np.save("training_data.npy", training_data)            
        
if __name__ == '__main__':
    Labeling.make_training_data()
    

[47, 52, 62, 46, 49, 47, 43, 49, 50, 44, 49, 39, 184, 113, 132, 94, 147, 208, 87, 143, 83, 136, 79, 105] 2088
2.2509578544061304
2.490421455938697
2.9693486590038316
2.203065134099617
2.346743295019157
2.2509578544061304
2.0593869731800765
2.346743295019157
2.3946360153256707
2.10727969348659
2.346743295019157
1.8678160919540232
8.812260536398467
5.411877394636015
6.321839080459771
4.501915708812261
7.040229885057471
9.961685823754788
4.166666666666666
6.8486590038314175
3.9750957854406135
6.513409961685824
3.7835249042145596
5.028735632183908


In [588]:
import torch
torch.cuda.is_available()
device = torch.device("cuda:0")

In [589]:
def reset_weights(m):
    '''
    Try resetting model weights to avoid
    weight leakage.
    '''
    for layer in m.children():
        if hasattr(layer, 'reset_parameters'):
            print(f'Reset trainable parameters of layer = {layer}')
            layer.reset_parameters()

In [736]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 64, 3)
        #self.conv2 = nn.Conv2d(8, 8, 5)
        #self.conv3 = nn.Conv2d(8, 8, 5)
        #self.conv4 = nn.Conv2d(8, 8, 5)
        self.conv5 = nn.Conv2d(64, 128, 3)
        
        x = torch.randn(52,300).view(-1,1,52,300)
        self._to_linear = None
        self.convs(x)
        self.fc1 = nn.Linear(self._to_linear, 48)
        self.fc2 = nn.Linear(48, 24)
        self.dropout = nn.Dropout(0.5)
        
    def convs(self, x):
        x = F.elu(self.conv1(x))
        #x = F.avg_pool2d(F.elu(self.conv1(x)), (2,2))
        
        #x = F.elu(self.conv2(x))
        #x = F.avg_pool2d(F.elu(self.conv2(x)), (2,2))
        
        #x = F.elu(self.conv3(x))
        #x = F.avg_pool2d(F.elu(self.conv3(x)), (2,2))
        
        #x = F.elu(self.conv4(x))
        #x = F.avg_pool2d(F.elu(self.conv4(x)), (2,2))
        
        x = F.avg_pool2d(F.elu(self.conv5(x)), (2,2))
        
        
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
            #print(x.shape)
            #print(self._to_linear)
        return x
        
    
    def forward(self, x):
        
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.elu(self.fc1(x))
        x = self.dropout(x)
        x = F.elu(self.fc2(x))
        return x


network = Net().to(device)
reset_weights(network)
print(network)
    

Reset trainable parameters of layer = Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Linear(in_features=454656, out_features=48, bias=True)
Reset trainable parameters of layer = Linear(in_features=48, out_features=24, bias=True)
Net(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=454656, out_features=48, bias=True)
  (fc2): Linear(in_features=48, out_features=24, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [735]:
from tqdm import tqdm
import numpy as np
import torch.optim as optim

#training_data = np.load("C:/spectsBS/training_data.npy", allow_pickle=True)
#print(training_data)

In [622]:
#import sys 
#import torch

#print(sys.getsizeof(training_data[0]))
#temp = torch.Tensor([training_data[0]]).view(-1,105,600)
#print(temp)
print(len(X))

1308


In [721]:
optimizer = torch.optim.SGD(network.parameters(), lr=0.0005, momentum = 0.9, weight_decay = 0.0001) 
loss_function = nn.CrossEntropyLoss()

In [2]:
#for batch in range(0, training_data, 32):
X = torch.Tensor([i[0] for i in training_data]).view(-1,52,300)


#print("cap")


In [625]:
import sys

print(sys.getsizeof(X[0]))
print(sys.getsizeof(training_data))
print(len(X[0]))

64
18664
52


In [626]:
X = X/255.0

In [627]:
y = torch.Tensor([i[1] for i in training_data])

print(len(X))
print(len(y))

2088
2088


In [628]:
VAL_PCT = 0.1  # lets reserve 10% of our data for validation
val_size = int(len(X)*VAL_PCT)

print(val_size)
train_X = X[:len(training_data)-val_size]
train_y = y[:len(training_data)-val_size]

print(len(train_X))
test_X = X[-val_size:]
test_y = y[-val_size:]

print(len(test_X))
print(len(test_y))

208
1880
208
208


In [725]:
BATCH_SIZE = 15
EPOCHS = 5

def train(net):
    
    for epoch in range(EPOCHS):
        for i in tqdm(range(0, len(train_X), BATCH_SIZE)): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
            #print(f"{i}:{i+BATCH_SIZE}")
            batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 52, 300)
            batch_y = train_y[i:i+BATCH_SIZE]
            
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            
            network.zero_grad()

            outputs = network(batch_X)
            
            #print(outputs)
            #print(outputs)
            batch_y = batch_y.long()
            #print(batch_y.shape)
            loss = loss_function(outputs, batch_y)
            loss.backward()
            optimizer.step()    # Does the update

        print(f"Epoch: {epoch}. Loss: {loss}")

In [733]:
train(network)



  0%|          | 0/126 [00:00<?, ?it/s]

Epoch: 0. Loss: 0.028388654813170433


  0%|          | 0/126 [00:00<?, ?it/s]

Epoch: 1. Loss: 0.022655632346868515


  2%|▏         | 2/126 [00:00<00:10, 12.26it/s]

Epoch: 2. Loss: 0.02779572084546089


  0%|          | 0/126 [00:00<?, ?it/s]

Epoch: 3. Loss: 0.025853430852293968


100%|██████████| 126/126 [00:25<00:00,  4.86it/s]


Epoch: 4. Loss: 0.017228757962584496


In [731]:
correct = 0
total = 0

network.eval()
test_X , test_y = test_X.to(device), test_y.to(device)
with torch.no_grad():
    for i in tqdm(range(len(test_X))):
        #real_class = torch.argmax(test_y[i])
        net_out = network(test_X[i].view(-1, 1, 52, 300))[0]  # returns a list, 
        
        predicted_class = torch.argmax(net_out)
        
        
        if predicted_class == test_y[i]:
            correct += 1
        total += 1
        

#8 epochs accuracy 25.743
#9 epochs accuracy 25.106
print("Accuracy: ", round((correct/total)*100, 3))



100%|██████████| 208/208 [00:00<00:00, 353.49it/s]

Accuracy:  7.212


In [472]:
print(training_data[3][0])
print(len(training_data[3][0][0]))

[[255 253 253 ... 252 253 228]
 [255 253 254 ... 253 252 225]
 [255 254 253 ... 253 252 224]
 ...
 [200 176  48 ...  47  53  47]
 [184 156  36 ...  38  37  35]
 [178 137  29 ...  25  17  31]]
300


In [732]:
#testing on the training data 

correct = 0
total = 0
batch = 10

#train_X , train_y = train_X.to(device), train_y.to(device)
with torch.no_grad():
    #for i in tqdm(range(len(train_X))):
    for i in tqdm(range(0, len(train_X), BATCH_SIZE)): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
        #print(f"{i}:{i+BATCH_SIZE}")
        batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 52, 300)
        batch_y = train_y[i:i+BATCH_SIZE]

        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        
        outs = network(batch_X)
        
        for count, j in enumerate(outs):
            temp = torch.argmax(j)
           
            if temp == batch_y[count] :
                #print(predicted_class, train_y[i])
                correct += 1
            total += 1
print("Accuracy: ", round((correct/total)*100, 3))

100%|██████████| 126/126 [00:04<00:00, 31.28it/s]

Accuracy:  90.479


In [13]:
print(training_data[1060][1])

tensor([11])


In [ ]:
print(training_data[5])


In [34]:
import torch
r = torch.rand(2,3)
t = r.view(-1, 3, 2)

print (r)
print(t)
print(t[0][0][0])

tensor([[0.6742, 0.2223, 0.9260],
        [0.3503, 0.8297, 0.1198]])
tensor([[[0.6742, 0.2223],
         [0.9260, 0.3503],
         [0.8297, 0.1198]]])
tensor(0.6742)


In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, ConcatDataset
from torchvision import transforms
from sklearn.model_selection import KFold

loss_function = nn.CrossEntropyLoss()
# For fold results
results = {}
  
# Set fixed random number seed
torch.manual_seed(42)


num_epochs = 25
k_folds = 10
# For fold results
results = {}
# Set fixed random number seed
torch.manual_seed(42)
# Define the K-fold Cross Validator
kfold = KFold(n_splits=k_folds, shuffle=True)


# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(kfold.split(training_data)):

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      training_data, 
                      batch_size=32, sampler=train_subsampler)
    #print(trainloader)
    #print(training_data)
    
    
    
    testloader = torch.utils.data.DataLoader(
                      training_data,
                      batch_size=32, sampler=test_subsampler)

    # Init the neural network
    network = Net().to(device)

    # Initialize optimizer
    #optimizer = torch.optim.Adam(network.parameters(), lr=1e-3, weight_decay = 0.0001)
    optimizer = torch.optim.SGD(network.parameters(), lr=0.0005, momentum = 0.9, weight_decay = 0.0001) 

    # Run the training loop for defined number of epochs
    for epoch in range(0, num_epochs):

        # Print epoch
        print(f'Starting epoch {epoch+1}')

        # Set current loss value
        current_loss = 0.0
        
        #print(len(trainloader))
        # Iterate over the DataLoader for training data
        for i, data in enumerate(trainloader, 0):
            data[0] = data[0].to(device)
            
            data[0] = data[0] / 255.0
            #print(data)
            #print("________________")
            # Get inputs
            
            #inputs, targets = data
            
            
            inputs = data[0].view(-1,1,105,600)
            targets = data[1]
            inputs = inputs.float()
            
            inputs, targets = inputs.to(device), targets.to(device)
            #print(inputs)
            #for cnt, k in enumerate (inputs):
                #print(k.shape)
                #inputs[cnt] = k.view(-1,105,600)
                #print(inputs[cnt].shape)
                      
            # Zero the gradients
            optimizer.zero_grad()
            #print(inputs.shape)
            #inpouts = torch.unsqueeze(inputs, 0)
            #print(inputs.view(10, 1, 105, 600).shape)
    
            # Perform forward pass
            outputs = network(inputs)
            
            #outputs = torch.argmax(outputs)
            #print(targets[0])
            #targets = targets.long()
            targets = targets.view(len(data[1]))
            
            #print(outputs.shape)
            
            # Compute loss
            loss = loss_function(outputs, targets)

            # Perform backward pass
            loss.backward()

            # Perform optimization
            optimizer.step()

            # Print statistics
            current_loss += loss.item()
            if i % 100 == 99:
                print('Loss after mini-batch %5d: %.3f' %
                      (i + 1, current_loss / 100))
                current_loss = 0.0
            
    # Process is complete.
    print('Training process has finished. Saving trained model.')

    # Print about testing
    print('Starting testing')

    # Saving the model
    #save_path = f'./model-fold-{fold}.pth'
    #torch.save(network.state_dict(), save_path)

    # Evaluationfor this fold
    
    correct, total = 0, 0
    with torch.no_grad():

      # Iterate over the test data and generate predictions
        for i, data in enumerate(testloader, 0):
            data[0] = data[0].to(device)
            data[0] = data[0] / 255.0
        # Get inputs
            inputs = data[0].view(-1,1,105,600)
            targets = data[1]
            inputs = inputs.float()

            inputs, targets = inputs.to(device), targets.to(device)
        # Generate outputs
            outputs = network(inputs)

        # Set total and correct
            
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            #correct += (predicted == targets).sum().item()
            cntr = 0
            for i in predicted:
                
                if i == targets[cntr]:
                    correct += 1
                cntr += 1
            #print((predicted == targets))
            #print((predicted == targets).sum())
            #print((predicted == targets).sum().item())
            #print((predicted == targets).sum().item())
            #print(total, correct)
            #print(predicted , targets)

      # Print accuracy
        print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
        print('--------------------------------')
        results[fold] = 100.0 * (correct / total)
        print (correct, total)
        print("targets", targets.size(0) , len(targets))

  # Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
sum = 0.0
for key, value in results.items():
    print(f'Fold {key}: {value} %')
    sum += value
print(f'Average: {sum/len(results.items())} %')



        

FOLD 0
--------------------------------
Starting epoch 1
Loss after mini-batch   100: 3.190
Loss after mini-batch   200: 3.181
Loss after mini-batch   300: 3.176
Loss after mini-batch   400: 3.177
Loss after mini-batch   500: 3.176
Loss after mini-batch   600: 3.177
Loss after mini-batch   700: 3.176
Loss after mini-batch   800: 3.175
Loss after mini-batch   900: 3.178
Loss after mini-batch  1000: 3.174
Starting epoch 2
Loss after mini-batch   100: 3.169
Loss after mini-batch   200: 3.171
Loss after mini-batch   300: 3.168
Loss after mini-batch   400: 3.166
Loss after mini-batch   500: 3.163
Loss after mini-batch   600: 3.164
Loss after mini-batch   700: 3.159
Loss after mini-batch   800: 3.155
Loss after mini-batch   900: 3.156
Loss after mini-batch  1000: 3.146
Starting epoch 3
Loss after mini-batch   100: 3.140
Loss after mini-batch   200: 3.131
Loss after mini-batch   300: 3.118
Loss after mini-batch   400: 3.110
Loss after mini-batch   500: 3.091
Loss after mini-batch   600: 3.086

In [30]:
print(training_data[1300])

[array(None, dtype=object), tensor([2])]
